In [ ]:
import cv2
import numpy as np
from moviepy import *
from stegano import lsb
from PIL import Image

In [ ]:
def str_splitter(main_str, substr_len):
    print(f'Number of sub-strings: {len(main_str)  // substr_len + (len(main_str) % substr_len != 0)}')
    return [main_str[i:i + substr_len] for i in range(0, len(main_str), substr_len)]


def get_video_attr(vid_path):
    vid = cv2.VideoCapture(vid_path)
    fourcc = int(vid.get(cv2.CAP_PROP_FOURCC))
    fps = vid.get(cv2.CAP_PROP_FPS)
    w = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
    list_frame = []  # shape(frame_count, height, width, rgb_val_count)
    count = 0
    while 1:
        success, image = vid.read()
        if not success:
            break
        list_frame.append(image)
        count += 1

    return list_frame, fourcc, fps, w, h

def encode_string_to_video(sec_file_path, cover_vid_path, stego_vid_path):
    # Encode file to a binary string
    with open(sec_file_path, 'rb') as f:
        binary_data = f.read()

    # Split the encoded strings to multiple substrings
    print(f'Your data is converted to a sequence of {len(binary_data)} bytes.')
    substr_len = int(input('How long should each sub-string be?: '))
    list_substr = str_splitter(binary_data, substr_len)

    # Read all necessary attributes from video
    list_frame, fourcc, fps, w, h = get_video_attr(cover_vid_path)

    # Hide data into video
    print(f'There are {len(list_frame)} frames in this video.')
    frame_hide_start = int(input(f'From which frame do you want to hide your data? (Choose between 0 and {len(list_frame) - len(list_substr) - 1}): '))
    frame_hide_end = frame_hide_start + len(list_substr)
    stego_vid = cv2.VideoWriter(stego_vid_path, fourcc, fps, (w, h))
    for i in range(len(list_frame)):
        if i in range(frame_hide_start, frame_hide_end):
            frame_pil = Image.fromarray(list_frame[i])
            secret_pil = lsb.hide(frame_pil, list_substr[i - frame_hide_start])
            new_frame = np.array(secret_pil)
            list_frame[i] = new_frame
        
        # list_frame[i] = cv2.cvtColor(list_frame[i], cv2.COLOR_BGR2RGB)
        stego_vid.write(list_frame[i])
    cv2.destroyAllWindows()
    stego_vid.release()

    # Export new video
    stego_vid = VideoFileClip(stego_vid_path)
    stego_vid = stego_vid.with_audio(VideoFileClip(cover_vid_path).audio)
    stego_vid.write_videofile(stego_vid_path, codec="libx264", audio_codec="aac")
    return binary_data

In [ ]:
def decode_video(stego_video_path, file_name):
    # Getting the encoded binary data
    binary_data = b''
    list_frame, _, _, _, _  = get_video_attr(stego_video_path)
    for frame in list_frame:
        print(frame.shape)
        frame_pil = Image.fromarray(frame)
        secret_data = lsb.reveal(frame_pil)
        if secret_data != None:
            binary_data += secret_data

    # Saving secret file
    with open(file_name, 'wb') as f:
        f.write(binary_data)

In [ ]:
binary_data = encode_string_to_video('methodEoF.py', r'Result\#1 bedwars trap.mp4', 'stego.mp4')

In [ ]:
decode_video('stego.mp4', 'methodEoF.py')

In [ ]:
vid = cv2.VideoCapture('L30_V001.mp4')
fourcc = int(vid.get(cv2.CAP_PROP_FOURCC))
fps = vid.get(cv2.CAP_PROP_FPS)
w = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(fourcc, fps, w, h)